In [1]:
import wilds
from wilds.datasets.waterbirds_dataset import WaterbirdsDataset
from torch.utils.data import Subset
import numpy as np

In [59]:
def majority_only_waterbirds_dataset(dataset:WaterbirdsDataset) -> WaterbirdsDataset:
    train_split_mask = dataset.split_array == dataset.split_dict["train"]
    train_split_idx = np.where(train_split_mask)[0]

    train_metadata_array = dataset.metadata_array[train_split_mask,:]

    #(train_metadata_array[:,0].numpy() == train_metadata_array[:,1].numpy()).mean()
    spurious = (train_metadata_array[:,0].numpy() == train_metadata_array[:,1].numpy())

    non_spurious_train_split_idx = train_split_idx[~spurious]

    dataset.split_array[non_spurious_train_split_idx] = -1

    return dataset

In [2]:
majority_only = True
dataset = WaterbirdsDataset(root_dir="datasets")
if majority_only:
    dataset = majority_only_waterbirds_dataset(dataset)
data_train = dataset.get_subset("train")
data_test  = dataset.get_subset("test")
data_valid = dataset.get_subset("val") 


NameError: name 'majority_only_waterbirds_dataset' is not defined

In [3]:
dataset.metadata_fields.index('background')

0

In [9]:
def f():
    for i in range(3):
        yield {"x":i , "y":i+1}

for x in f():
    print(x)

{'x': 0, 'y': 1}
{'x': 1, 'y': 2}
{'x': 2, 'y': 3}


In [1]:
import torch 
saved_data_train_path = "./datasets/office_home_train.pt"
data_train = torch.load(saved_data_train_path)


In [38]:
from torch.utils.data import DataLoader, Dataset, random_split, Subset
import random

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)


g = torch.Generator()
g.manual_seed(42)

test_dl = DataLoader(data_train, batch_size=512, worker_init_fn=seed_worker, generator=g,shuffle=False)

def dl_to_sampler(dl):
    dl_iter = iter(dl)
    def sample():
        nonlocal dl_iter
        try:
            return next(dl_iter)
        except StopIteration:
            dl_iter = iter(dl)
            return next(dl_iter)
    return sample

perturb_sampler = dl_to_sampler(test_dl)



In [10]:
from torchvision.models import resnet18
import torch.nn as nn

model = nn.Linear(3,4)

In [31]:
from collections import defaultdict

a =  defaultdict(dict)

In [28]:
torch.cat((None,a)).size()

TypeError: expected Tensor as element 0 in argument 0, but got NoneType

In [19]:

from transformers import AutoImageProcessor, ViTForImageClassification

model = ViTForImageClassification.from_pretrained("facebook/dino-vitb16")


Some weights of ViTForImageClassification were not initialized from the model checkpoint at facebook/dino-vitb16 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
model.classifier(torch.ones((1,768)))

tensor([[0.7842, 0.4327]], grad_fn=<AddmmBackward0>)

In [4]:
import torch

a = torch.load("/datasets/home/hbenoit/D-BAT-exp/test_precompute/waterbird/perturb=ood_is_test/resnet50_pretrained=True/ep[2]/ep=[2]_lrmax=0.001_alpha=0.1_dataset=waterbird_model=resnet50_pretrained=True_scheduler=none_seed=0_opt=sgd_ensemble_size=2_no_diversity=False/probs_perturb_batch0.tmp")

a.device

device(type='cpu')

In [74]:
path = "/datasets/home/hbenoit/D-BAT-exp/test_precompute/waterbird/perturb=ood_is_test/resnet50_pretrained=True/ep[2]/ep=[2]_lrmax=0.001_alpha=0.1_dataset=waterbird_model=resnet50_pretrained=True_scheduler=none_seed=0_opt=sgd_ensemble_size=2_no_diversity=False"

import glob
import os

list(glob.glob(os.path.join(path, "probs_perturb*.tmp")))
os.listdir(path)

['ckpt.pt',
 'probs_perturb_batch2.tmp',
 'probs_perturb_batch3.tmp',
 'probs_perturb_batch4.tmp',
 'probs_perturb_batch5.tmp',
 'probs_perturb_batch6.tmp',
 'probs_perturb_batch7.tmp',
 'probs_perturb_batch8.tmp',
 'probs_perturb_batch9.tmp',
 'probs_perturb_batch0.tmp',
 'probs_perturb_batch1.tmp']

In [72]:
os.path.join(path, "probs_perturb*")

'/datasets/home/hbenoit/D-BAT-exp/test_precompute/waterbird/perturb=ood_is_test/resnet50_pretrained=True/ep[2]/ep=[2]_lrmax=0.001_alpha=0.1_dataset=waterbird_model=resnet50_pretrained=True_scheduler=none_seed=0_opt=sgd_ensemble_size=2_no_diversity=False/probs_perturb*'

In [69]:
a = torch.zeros((3, 52, 65))
b= torch.ones((52,65)).unsqueeze(0)

torch.cat((a,b))[0].size()

torch.Size([52, 65])

In [40]:
j=-1
for i in range(20):
    j = (j+1)%18
    print(j)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
0
1


In [39]:
from collections import defaultdict
a= defaultdict(list)
for i in range(2):
    for j in range(18):
        item = perturb_sampler()[0][0][0][0][0].item()
        a[i].append(item)

a[0] == a[1]

True

In [5]:
i=0
for batch in test_dl:
    print("hello")
    i+=1

i

hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello


18

In [33]:
data_test.tensors[0].size()

torch.Size([4357, 3, 90, 90])

In [48]:
from torch import default_generator, randperm
indices = randperm(4357, generator=torch.Generator().manual_seed(42)).tolist()

data_test.tensors[0][indices[1900:]].size()

torch.Size([2457, 3, 90, 90])

In [32]:
torch.cat((data_train.tensors[1],data_test.tensors[1])).size()

torch.Size([13161])

In [58]:
import numpy as np
import torch
np.sum(tensor([ True,  True,  True, False, False, False]))


NameError: name 'tensor' is not defined

In [25]:
def check_stats(data):

    spurious = (data.metadata_array[:,0].numpy() == data.metadata_array[:,1].numpy())
    confounder_strength = spurious.mean()

    print(f"Confounder_strength = {confounder_strength:.3f}")
    
    return confounder_strength



In [37]:
def majority_only(data):

    spurious = (data.metadata_array[:,0].numpy() == data.metadata_array[:,1].numpy())

    spurious_idx= np.arange(len(data))[spurious]

    return data[spurious_idx]
